**School of Mechanical Engineering - State University of Campinas**
Paulo Yoshio Kuga
First Release: October 2023

Present version: 20231221

this is the main program to run neural network identification 

In [10]:
using LinearAlgebra
using ControlSystems
using Plots
using ControlSystemIdentification
using Random
using Statistics
using DelimitedFiles
using CUDA
using Flux
using Hyperopt
using FluxArchitectures


include("KugaPack.jl")
include("sysid.jl")
include("blockpack.jl")
include("analysis.jl")
CUDA.allowscalar(true)

In [11]:
#numerical conditions are set up
dt = 5e-3;
maxT = 5;
ns = round(Int,maxT/dt)+1;
t = 0:dt:maxT;

ns

1001

In [12]:
#problem conditions are set up
x0 = [0; 0; 0; 0;];

n_frac = round(Int,ns/4)

w = 80
U1 = chirp(w,0:dt:maxT/4-dt)
U2 = PulseGen(n_frac+n_frac+1)
U3 = rand(1,n_frac).-0.5;


U = vec([U1; U2'; U3']);

In [4]:
png(
    plot(
    t, 
    U, 
    labels="Entrada",
    ylabel = "Força (N)", 
    xlabel = "Tempo",
    title  = "Esforço U aplicado",
),"grafico_u");

In [4]:
#using the given parameters, the system (discrete space state) is created 
Param = [0.77 0.59 2.1 1.2 9 200 200 200];
sysP,sysD = F2DOF(dt,Param);

Y,_,_= lsim(sysD,U',t,x0=x0); #time response
magD,phaseD,w = bode(sysD); #frequency response

In [6]:
png(
    plot(
    t, 
    Y', 
    labels=["Massa 1" "Massa 2"],
    ylabel = "Posição (m)", 
    xlabel = "Tempo",
    title  = "Deslocamento das massas",
),"grafico_Y_U");

In [5]:
#identification parameters
na = 4;
nb = 1;
ny = 2;
nu = 1;
p = 25;

In [9]:
Data = SimpleData(Y,U',ns)
Matrix_1 = Testador(Data,nu,ny,30,30,1)


writedlm( "fuapenas.csv",  Matrix_1, ';')

png(
    plot(
    heatmap(Matrix_1), 
    ylabel = "Largura da Rede", 
    xlabel = "Profundidade da Rede",
    title  = "Erro na Identificação - Rede análoga ao sistema",
),"rede_fuapenas");

minimum(Matrix_1) #0.7992987195086019

0.7992987195086019

In [17]:
Data = DataNARX(Y,U', ny, nu, na, nb,ns)
Matrix_2 = Testador(Data,nu*nb+ny*na,ny,30,30,1)


writedlm( "NARX.csv",  Matrix_2, ';')

png(
    plot(
    heatmap(Matrix_2), 
    ylabel = "Largura da Rede", 
    xlabel = "Profundidade da Rede",
    title  = "Erro na Identificação - Rede análoga ao sistema",
),"NARX");

minimum(Matrix_2) #0.8363667320144382

0.8363667320144382

In [23]:
detail = 5:30

png(
    plot(
        heatmap(detail,detail,Matrix_1[detail,detail]), 
    ylabel = "Largura da Rede", 
    xlabel = "Profundidade da Rede",
    title  = "Erro na Identificação - Rede análoga ao sistema",
),"detalhamento_redepadrao");

png(
    plot(
        heatmap(detail,detail,Matrix_2[detail,detail]), 
    ylabel = "Largura da Rede", 
    xlabel = "Profundidade da Rede",
    title  = "Erro na Identificação - Rede análoga ao sistema",
),"detalhamento_NARX");

In [35]:
V = Int.([nu; 24*ones(6); ny])
Yl = SimpleOpt(V,SimpleData(Y,U',ns),relu)
png(
    plot(
    t, 
    [Y' Yl'], 
    labels=["Massa 1" "Massa 2" "Massa 1 - estimado" "Massa 2 - estimado"],
    ylabel = "Posição (m)", 
    xlabel = "Tempo",
    title  = "Deslocamento das massas",
),"identificação_fu_best");

In [9]:
V = Int.([nu*nb+ny*na; 23*ones(7); ny])
Yl = SimpleOpt(V,DataNARX(Y,U', ny, nu, na, nb,ns),relu)
png(
    plot(
    t, 
    [Y' Yl'], 
    labels=["Massa 1" "Massa 2" "Massa 1 - estimado" "Massa 2 - estimado"],
    ylabel = "Posição (m)", 
    xlabel = "Tempo",
    title  = "Deslocamento das massas",
),"identificação_NARX_best");